<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_3_Reentrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introducción

La aceleración es la tasa de variación de la velocidad de un objeto cuando se mueve. Si el objeto en cuestión mantiene su velocidad de forma constante, entonces no está acelerando, algo que sí hará cuando su velocidad cambia. La aceleración se mide en metros por segundo al cuadrado, y lo hace en base al tiempo que le lleva pasar de una velocidad a otra, o bien en una fuerza que se aplica sobre el objeto. [1]

Se puede calcular la aceleración de un cuerpo mediante la siguiente fórmula:

## <center>$\frac{vf-vi}{tf-ti} = {a}$

A continuación, calcularemos la aceleración de varios cuerpos utilizando OpenMP [5].

#2 Armado del ambiente



##2.1  Generar code_acel.cpp

Mediante código Python, genero una cadena de texto con código C. Luego, guardo la cadena en el archivo code_acel.cpp

In [ ]:
code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h> //Cabecera OpenMP

//------------------------------------------------
// Macros para medir el tiempo

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL     1
#define HTH_ACEL_SEC  2
#define HTH_ACEL_OMP  3

//---------------------------------------------------------

int main(int argc, char*argv[])
{
  int i,c;
  TIEMPO_INI ( HTH_TOTAL )

  //Leo los parametros.
  if (argc != 4 )
  {
    std::cerr<<"Error al ingresar los parámetros" <<std::endl;
    exit( - 1 );
   }

   int time               =  atoi( argv[1] );
   int cantidadVehiculos  =  atoi( argv[2] );
   int ciclos             =  atoi( argv[3] );

   if ( time <= 0)
   {
      std::cerr<<"El tiempo debe ser mayor a 0 [Parametro 1]" <<std::endl;
      exit( - 2 );
   }

   if ( cantidadVehiculos <= 0)
   {
      std::cerr<<"La cantidad de vehículos debe ser mayor a 0 [Parametro 2]" <<std::endl;
      exit( - 3 );
   }

   if ( ciclos <= 0)
   {
      std::cerr<<"La cantidad de ciclos debe ser mayor a 0 [Parametro 3]" <<std::endl;
      exit( - 4 );
   }

   std::cout<<"Tiempo Acel sec : " <<time<< " [ms]"<<std::endl;

   //-----------------------------------------------------
   // Defino la memoria de los vectores para la velocidad inicial y final

   std:: vector<double>   VI ( cantidadVehiculos );
   std:: vector<double>   VF ( cantidadVehiculos );
   std:: vector<double>   AC ( cantidadVehiculos );

   for (int i = 0; i < cantidadVehiculos; i++)
   {
      VI[i] = (rand()/(double)RAND_MAX)*100.0; // 
      VF[i] = (rand()/(double)RAND_MAX)*100.0; //
   }

  //------------------------------------------
  // Realizo el cálculo de la aceleración en forma secuencial

  TIEMPO_INI ( HTH_ACEL_SEC )

  for(c=0; c<ciclos; c++)
  {
    for(i=0; i< cantidadVehiculos ; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_SEC )

  // ---------------------------------------------
  // Realizo el cálculo de la aceleración con OpenMP

  TIEMPO_INI ( HTH_ACEL_OMP )

  for(c=0; c<ciclos; c++)
  {
    #pragma omp parallel for
    for(i=0; i< cantidadVehiculos ; i++)
    {
      AC[i] = (VF[i] - VI[i]) / time;
    }
  }

  TIEMPO_FIN ( HTH_ACEL_OMP )

  //------------------------------------------------

  std::cout<<"Valores Reales  : " <<std::endl;
  std::cout<<"Tiempo Acel sec : " <<TIEMPO_GET(HTH_ACEL_SEC)<< " [ms]"<<std::endl;
  std::cout<<"Tiempo Acel Omp : " <<TIEMPO_GET(HTH_ACEL_OMP)<< " [ms]"<<std::endl;
  std::cout<<std::endl;  
}
"""

text_file = open("code_acel.cpp","w")
text_file.write(code)
text_file.close()



##2.2 Compilación de código C (cálculo de aceleración)

In [ ]:
!g++ -o acel -fopenmp code_acel.cpp

#3 Desarrollo

In [ ]:
%env OMP_NUM_THREADS=2

# --------------------------------------------
from datetime import datetime
tiempo_total = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#----------------------------------------------------------------------
try:

#---- Ejecución

# Parámetros:
# 1 -> Tiempo en segundos
# 2 -> Cantidad de vehículos
# 3 -> Cantidad de ciclos

  !./acel 10 1000000 50
#----

  tiempo_total = datetime.now() - tiempo_total;
  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )

except Exception as e:
  print("Error:",e.args)

env: OMP_NUM_THREADS=2
Tiempo Acel sec : 10 [ms]
Valores Reales  : 
Tiempo Acel sec : 510.522 [ms]
Tiempo Acel Omp : 466.601 [ms]

Tiempo Total:  1121.679 [ms]


#4 Tabla de pasos



 Procesador | Función | Detalle
------------|---------|----------
CPU      |  open("code_acel.cpp","w") | Creo el archivo code_acel.cpp
CPU      |  text_file.write(code)     | Escribo el código C en el archivo code_acel.cpp.
CPU      |text_file.close()       | Cierro el archivo.
         |  !g++ -o acel -fopenmp code_acel.cpp                | Compilación del código C.
CPU      |  datetime.now()        | Toma el tiempo actual.
         |  !./acel 10 1000000 50| Ejecución del cálculo de aceleración en openMP. 
CPU      |  datetime.now()        | Toma el tiempo final.
CPU      |  print()               | Informa los resultados.

#5 Conclusiones

Podemos observar que los tiempos para calcular la aceleración secuencial son mayores que al calcular la aceleración en paralelo (OpenMP).

Tiempo Acel sec : 608.134 [ms] <br>
Tiempo Acel Omp : 460.362 [ms] <br>
Tiempo Total:     1219.696 [ms]

Para la ejecución secuencial, se utilizo un FOR para recorrer los vectores que contienen la velocidad inicial y final de los cuerpos.
Para la ejecución en paralelo se utiliza OpenMP. La sentencia #pragma omp parallel for, nos indica que la parte de código que la comprende puede ser ejecutada por varios hilos. Se crea una tarea implícita para cada hilo perteneciente al equipo de hilos creado por el parallel.(Por defecto, openmp sabe que vamos a iterar por el valor i). 

El punto más relevante en este ejercicio es demostrar el funcionamiento y utilizacion de OpenMP.

En cuanto a lecciones aprendidas, comprendí el funcionamiento de OpenMP. <br>
Como mejora, se podrían agregar métricas de ejecución para un posterior análisis.

#6 Bibliografía

[1] Aceleracion: [Referencia](https://www.fisicalab.com/apartado/aceleracion)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)

[4] Sliders Parametros: [Referencia](https://colab.research.google.com/notebooks/forms.ipynb)

[5] openMP: [Enlace](https://es.wikipedia.org/wiki/OpenMP)